In [1]:
!git clone https://github.com/AlinaGalenina/ML_HOMEWORK3.git
!wget https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
!wget  https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz

Cloning into 'ML_HOMEWORK3'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 1), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.
--2020-11-23 16:38:02--  https://snap.stanford.edu/data/loc-gowalla_edges.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6351523 (6.1M) [application/x-gzip]
Saving to: ‘loc-gowalla_edges.txt.gz’

loc-gowalla_edges.t 100%[===================>]   6.06M  1.80MB/s    in 5.1s    

2020-11-23 16:38:08 (1.18 MB/s) - ‘loc-gowalla_edges.txt.gz’ saved [6351523/6351523]

--2020-11-23 16:38:08--  https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|17

In [2]:
!gunzip loc-gowalla_edges.txt.gz
!gunzip loc-gowalla_totalCheckins.txt.gz

In [3]:
from ML_HOMEWORK3.src.ap_core import Ap
from ML_HOMEWORK3.src.ap_data import load_edges, load_checkins
import numpy as np
from tqdm import tqdm

ds_path = "loc-gowalla_edges.txt"
chk_path = "loc-gowalla_totalCheckins.txt"

In [7]:
def get_intersect(gt, pred):
    gt = set(gt)
    pred = set(pred)
    return len(gt.intersection(pred))


csr_ds = load_edges(ds_path)
checkins_train, checkins_test = load_checkins(chk_path)

In [5]:
ap = Ap()
ap.fit(csr_ds, 10)

100%|██████████| 10/10 [02:27<00:00, 14.73s/it]


In [8]:
topn_base = ap.predict(checkins_train)
cluster_prec = 0
base_prec = 0
norm = len(np.unique(checkins_test.user_id)) * 10
for user_id in tqdm(np.unique(checkins_test.user_id)):
    gt_locs = checkins_test.loc[checkins_test.user_id == user_id, "location_id"].values
    if user_id in checkins_train.user_id:
        train_locs = ap.predict(checkins_train, user_id)
        cluster_prec += get_intersect(gt_locs, train_locs)
        base_prec += get_intersect(gt_locs, topn_base)
        # test_locs = ap.predict(checkins_test, user_id)



100%|██████████| 1071/1071 [00:10<00:00, 105.91it/s]


In [9]:
print(f"Base AP {base_prec/norm}")
print(f"Cluster AP {cluster_prec/norm}")


Base AP 0.013445378151260505
Cluster AP 0.2859943977591036
